# 8.5 감성 분석

In [1]:
import numba as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 지도학습 기반 감성 분석 실습 - IMDB 영화평

In [4]:
reviwe_df = pd.read_csv('./data/labeledTrainData.tsv', header=0, sep="\t", quoting=3)
reviwe_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [5]:
reviwe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [6]:
print(reviwe_df['review'][0][:1000])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [7]:
reviwe_df['review'] = reviwe_df['review'].str.replace('<br />', ' ')

In [8]:
print(reviwe_df['review'][0][:1000])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.  Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.  The actual feature film bit when it finally starts is only on for 2

In [9]:
import re
reviwe_df['review'] = reviwe_df['review'].apply(lambda x : re.sub("[^a-zA-z]", " ", x))

In [10]:
print(reviwe_df['review'][0][:1000])

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

In [9]:
from sklearn.model_selection import train_test_split

class_df = reviwe_df['sentiment']
feature_df = reviwe_df.drop(['id', 'sentiment'], axis=1, inplace=False)

x_train, x_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

x_train.shape, x_test.shape

((17500, 1), (7500, 1))

- CountVectorizer 

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cvect = CountVectorizer(stop_words='english', ngram_range=(1,2))
x_train_cvect = cvect.fit_transform(x_train['review'], y_train)

In [12]:
from sklearn.linear_model import LogisticRegression


In [13]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression( C=10)
lr_clf.fit(x_train_cvect,y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
x_test_cvect = cvect.transform(x_test['review'])
pred = lr_clf.predict(x_test_cvect)

In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.8861333333333333

- TfidfVectorizer

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
tvect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
x_train_tvect = tvect.fit_transform(x_train['review'], y_train)

In [18]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression( C=10)
lr_clf.fit(x_train_tvect,y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
x_test_tvect = tvect.transform(x_test['review'])
pred = lr_clf.predict(x_test_tvect)

In [20]:
accuracy_score(y_test, pred)

0.8934666666666666

- 파이프라인

In [24]:
pipeline = Pipeline([
    ('tfidf_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

pipeline.fit(x_train['review'], y_train)
pred = pipeline.predict(x_test['review'])
pred_probs = pipeline.predict_proba(x_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test,pred),
                                                roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8861, ROC-AUC는 0.9502


In [23]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

pipeline.fit(x_train['review'], y_train)
pred = pipeline.predict(x_test['review'])
pred_probs = pipeline.predict_proba(x_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test,pred),
                                                roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8935, ROC-AUC는 0.9598
